In [1]:
!git clone https://github.com/soCzech/TransNetV2.git

fatal: destination path 'TransNetV2' already exists and is not an empty directory.


In [2]:
!python3 -m pip install --upgrade pip
!add-apt-repository ppa:jonathonf/ffmpeg-4 -y
!apt-get update
!apt-get install -y ffmpeg
!apt-get install -y libavcodec-extra


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1581 B]
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease              
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease              
Hit:6 http://ppa.launchpad.net/jonathonf/ffmpeg-4/ubuntu bionic InRelease      
Err:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done
W: GPG error: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu18

In [3]:
!pip install ffmpeg-python pillow

In [4]:
! cd TransNetV2 && python3 setup.py install

running install
running bdist_egg
running egg_info
writing transnetv2.egg-info/PKG-INFO
writing dependency_links to transnetv2.egg-info/dependency_links.txt
writing entry points to transnetv2.egg-info/entry_points.txt
writing top-level names to transnetv2.egg-info/top_level.txt
reading manifest file 'transnetv2.egg-info/SOURCES.txt'
writing manifest file 'transnetv2.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/transnetv2
creating build/bdist.linux-x86_64/egg/transnetv2/transnetv2-weights
copying build/lib/transnetv2/transnetv2-weights/saved_model.pb -> build/bdist.linux-x86_64/egg/transnetv2/transnetv2-weights
creating build/bdist.linux-x86_64/egg/transnetv2/transnetv2-weights/variables
copying build/lib/transnetv2/transnetv2-weights/variables/variables.data-00000-of-00001 -> build/bdist.linux-x86_64/egg/transnetv2/transnetv2-weights/variable

In [5]:

!pip install tensorflow[and-cuda]==2.1.1

In [6]:
from transnetv2 import TransNetV2

In [7]:
model = TransNetV2()

[TransNetV2] Using weights from /usr/local/lib/python3.6/dist-packages/transnetv2-1.0.0-py3.6.egg/transnetv2/transnetv2-weights/.


In [8]:
video = "cvpr08demo.mpg"

In [9]:
video_frames, single_frame_predictions, all_frame_predictions = model.predict_video(video)

[TransNetV2] Extracting frames from cvpr08demo.mpg
[TransNetV2] Processing video frames 3103/3103


In [10]:
video_frames.shape

(3103, 27, 48, 3)

In [11]:
single_frame_predictions.shape

(3103,)

In [12]:
all_frame_predictions.shape

(3103,)

In [13]:
all_frame_predictions

array([3.9849427e-04, 4.8510102e-04, 4.5428865e-04, ..., 9.4565642e-01,
       8.2267410e-01, 3.5310686e-01], dtype=float32)

In [14]:
scenes = model.predictions_to_scenes(single_frame_predictions)

In [15]:
scenes.tolist()

[[0, 156],
 [158, 511],
 [512, 528],
 [529, 558],
 [559, 737],
 [738, 1389],
 [1390, 1804],
 [1805, 1827],
 [1828, 1897],
 [1898, 2369],
 [2370, 2579],
 [2580, 2824],
 [2825, 2981],
 [2982, 3034],
 [3035, 3094],
 [3095, 3102]]

In [18]:
import os
import ffmpeg
def cut_video(video_path, frame_ranges):
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    output_dir = os.path.join(os.path.dirname(video_path), base_name + "TransnetV2")
    os.makedirs(output_dir, exist_ok=True)
    probe = ffmpeg.probe(video_path)
    video_streams = [stream for stream in probe['streams'] if stream['codec_type'] == 'video']
    frame_rate = eval(video_streams[0]['r_frame_rate'])
    
    for i, (start_frame, end_frame) in enumerate(frame_ranges):
        start_time = start_frame / frame_rate
        end_time = (end_frame + 1) / frame_rate
        output_file = os.path.join(output_dir, f"{base_name}_part_{i+1}.mp4")
        
        ffmpeg.input(video_path, ss=start_time, to=end_time).output(output_file).run()

    print(f"All video parts have been saved in the folder: {output_dir}")

In [19]:
cut_video(video, scenes)

All video parts have been saved in the folder: cvpr08demoTransnetV2
